# **Estruturas criptograficas: TP2 problema 3**

In [9]:
import hashlib, os
from sage.all import *

In [10]:

# # Geração dos primos  q, p
# bq     = 192                # tamanho em bits do primo "q"
# bp     = 768                # tamanho minimo em bits do primo "p"

# q = random_prime(2^bq-1,lbound=2^(bq-1))

# t = q*3*2^(bp - bq)
# while not is_prime(t-1):
#     t = t << 1

# p = t - 1

# # Aneis e Corpos
# Fp     = GF(p)                  # corpo primo com "p" elementos
# R.<z>  = Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
# f     = R(z^2 + z + 1)
# Fp2.<z> = GF(p^2, modulus=f)   
# # extensão de Fp de dimensão 2 cujo módulo é o polinómio "f"
# # o polinómio "f"  é irredutivel, tem grau 2 e verifica  z^3 = 1 mod f
# # se o ponto (x,y) verificar a equação y^2 = x^3 + 1, 
# #      então o ponto (z*x,y) verifica a mesma equação

# def trace(x):       # função linear que mapeia Fp2  em  Fp
#     return x + x^p

# # Curvas Elipticas supersingulares em Sagemath

# # a curva supersingular sobre Fp2  definida pela equação  y^2 = x^3 + 1
# E2 = EllipticCurve(Fp2, [0,1])

# # ponto arbitrário  de ordem "q" em E2        
# cofac = (p + 1)//q
# G = cofac * E2.random_point()

# # emparelhamento e oraculo DDHP

# def phi(P):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
#     (x,y) = P.xy()
#     return E2(z*x,y)

# def TateX(P,Q,l=1):      # o emparelhamento de Tate generalizado
#     return P.tate_pairing(phi(Q), q, 2)^l

# def ddhp(P,Q,R):        # o oraculo DDHP  que decide se (P,Q,R) é um triplo de DH
#     return tateX(P,Q) == tateX(R,G)

In [11]:
class BF:
    
    def __init__(self):
        self.version = 2
        self.n = 1024
        self.n_p = 512
        self.n_q = 160
        self.hashfnc = hashlib.sha1

        
 
    def BF_setup(self):
        q = random_prime(2^self.n_q-1,lbound=2^(self.n_q-1))
                
        # print("q:" + str(q))
        
        #NOTE: verificar se está certo
        t = q*3*2^(self.n_p - self.n_q)
        while not is_prime(t-1):
            t = t << 1

        p = t - 1
        
        # print("p:" + str(p))
        
        Fp     = GF(p)                  # corpo primo com "p" elementos
        R.<z>  = Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
        f     = R(z^2 + z + 1)
        Fp2.<z> = GF(p^2, modulus=f)   
        E2 = EllipticCurve(Fp2, [0,1])

        # ponto arbitrário  de ordem "q" em E2        
        cofac = (p + 1)// q
        G = cofac * E2.random_point()
        # print("G:" + str(G.xy()))
        
        s = 0
        while s <= 2:
            nounce = os.urandom(16)
            s = self.hashfnc(nounce).digest()
            s = int.from_bytes(s, byteorder='big')
            s = s % q
            
        # print("s:" + str(s))
        
        G_pub = s * G
        
        # print("G_pub:" + str(G_pub.xy()))
        
        return (E2, p, q, G, G_pub), s
    
    def HashToPoint(self, E, p, q, id):
        y = self.HashToRange(id, p)
        x = pow((y^2 - 1),((2 * p - 1) // 3), p)
        
        Q_ = E.point((x, y))
        
        Q = ((p + 1) // q) * Q_
        
        return Q
        
    def HashToRange(self, s, n):
        hashlen = self.hashfnc().digest_size
        v = [0, 0, 0]
        h = ['', '', '']
        
        v[0] = 0
        h[0] = b'\x00' * hashlen
        for i in range(1, 3):
            t = h[i-1] + s.encode()
            h[i] = self.hashfnc(t).digest()
            a = int.from_bytes(h[i], byteorder='big')
            v[i] = (256**hashlen) * v[i-1] + a

        v = v[i] % n
        return v
    
    def KeyGen(self, E, p, q, id):
        Q = self.HashToPoint(E, p, q, id)
        
        return Q
    
    def KeyExtract(self, E, p, q, id, s):
        d = self.HashToPoint(E, p, q, id)
        
        key = s * d

        return key    
    
    def phi(self, p, P, E2):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
        Fp     = GF(p)                  
        R.<z>  = Fp[]                   
        f     = R(z^2 + z + 1)
        Fp2.<z> = GF(p^2, modulus=f) 
        (x,y) = P.xy()
        return E2(z*x,y)
    
    #NOTE: verificar se está certo verificar o l
    def TateX(self, p, q, P, Q, E2, l=1):      # o emparelhamento de Tate generalizado
        return P.tate_pairing(self.phi(p, Q, E2), q, 2)^l 

        
        
    def Encrypt(self, E, p, q, G, G_pub, id, m):
        pass
        
        
        
        
        
        
        
    
    

In [12]:
bf = BF()
public_params, s = bf.BF_setup()

d = bf.KeyGen(public_params[0], public_params[1], public_params[2], "Universidade do Minho")
print("d:" + str(d.xy()))

key = bf.KeyExtract(public_params[0], public_params[1], public_params[2], "Universidade do Minho", s)
print("key:" + str(key.xy()))

# bf.Encrypt(public_params[0], public_params[1], public_params[2], public_params[3], public_params[4], "Universidade do Minho", "SageMath ser merda")

d:(133605862028571868603277797973270280052548071788591885969274127756988994379618355030468900747860585050037227124962070162562358803265988519052563350454075633551380445573552722636772109, 5683022209363294289231377449169329056245299589819125502974334991016871701056112633414519443329203004059906320188765391754588990906553402733070404772175210282087518932069585842227737)
key:(83023178120881654664222553142023324688934686044752817978591928016361975084560933686219426095177866729506156457466802630004451322918978906597879455887431041544941526807481784943779502, 112889591209168756526374803761206447297251265124003399281851879240618904493583790640416142359222057108373143451976462039927013211054737285770807306741677625149360884916661783976425875)


In [13]:
# def Encrypt(self, E, p, q, G, G_pub, id, m):
#         hashlen = self.hashfnc().digest_size
#         print("hashlen:" + str(hashlen))
        
#         Q_id = self.HashToPoint(E, p, q, id)
        
#         rho = os.urandom(16)
#         rho = self.hashfnc(rho).digest()
        
#         print("rho:" + str(len(rho)))
        
#         t = self.hashfnc(m.encode()).digest()
        
#         print("t:" + str(len(t)))
        
#         l = self.HashToRange(str(rho + t), q)  
        
#         print(l)
        
#         U = l * G
        
#         print("U:" + str(U.xy()))
        
#         theta = self.TateX(p, q, G_pub, Q_id, E)

#         print("theta:" + str(theta))
        
#         theta_ = pow(theta, l)
        
#         print("theta_:" + str(theta_))
        